Prior to running this notebook, change the following values:

In [ ]:
# Location of the file containing the scope. Format should be IP or CIDR, each on a new line.
ipaddresses_file = "./scope.txt"

# IPInfo.io Access Token (If you don't have one, check out https://ipinfo.io/signup )
ipinfo_access_token = ""

The bulk of the information gathering is done below, and will result in a table of IP address with information. Further below, a geographical map is provided as well.

In [ ]:
import ipaddress

ips_raw=[]
ips_info=[]

# Read in the ipaddresses file, storing the contents so we can close it soonest
with open(ipaddresses_file,'r') as f:
    ips_raw = f.read().splitlines()

# Get the general IP information, and split CIDRs
for ip in ips_raw:
    try:
        # Treat each entry as a "network"; those without CIDR will still work as a single IP
        ip_network = ipaddress.ip_network(ip)

        for host in ip_network.hosts():
            ip_temp={}
            try:
                ip_temp["address"] = format(host)
                ip_temp["address-details"] = []
                ip_temp["is-global"] = False

                if host.is_global:
                    ip_temp["address-details"].append("Global IP")
                    ip_temp["is-global"] = True
                if host.is_private:
                    ip_temp["address-details"].append("Private IP")
                if host.is_reserved:
                    ip_temp["address-details"].append("Reserved IP")
                if host.is_unspecified:
                    ip_temp["address-details"].append("Unspecified IP")
                if host.is_link_local:
                    ip_temp["address-details"].append("Link-Local IP")
                if host.is_loopback:
                    ip_temp["address-details"].append("Loopback IP")

                # Add it to the rest
                ips_info.append(ip_temp)
            except:
                pass
    except:
        pass

# Get the IPInfo.io information for Global IPs
import ipinfo

ipinfo_handler = ipinfo.getHandler(ipinfo_access_token)

for ip in ips_info:
    if not ip_temp["is-global"]:
        continue

    ipinfo_details = ipinfo_handler.getDetails(ip["address"])

    ip["hostname"] = ipinfo_details.all.get("hostname","(?)")
    ip["organization"] = ipinfo_details.all.get("org","(?)")
    ip["location"] = f"{ipinfo_details.city}, {ipinfo_details.region} {ipinfo_details.country}"
    ip["latitude"] = ipinfo_details.latitude
    ip["longitude"] = ipinfo_details.longitude

In [ ]:
# Print the IP data in a table
import pandas as pd

tbl = pd.DataFrame({
    'IP': [x["address"] for x in ips_info],
    'Details': [", ".join([y for y in x["address-details"]]) for x in ips_info],
    'Organization': [x["organization"] for x in ips_info],
    'Location': [x["location"] for x in ips_info],
    'Hostname': [x["hostname"] for x in ips_info]
})

print(f"Information for {len(ips_info)} IP address:")

pd.set_option('display.max_rows', None)
tbl

In [ ]:
# Plot the IP geolocation on a map
from ipyleaflet import Map, Marker

ip_map = Map()
locations = {}

# Bounds
lat_highest = -180.0
lat_lowest  = 180.0
lon_highest = -180.0
lon_lowest  = 180.0

for ip in ips_info:
    try:
        lat = float(ip["latitude"])
        lon = float(ip["longitude"])
        
        if lat > lat_highest:
            lat_highest=lat
        elif lat < lat_lowest:
            lat_lowest=lat
            
        if lon > lon_highest:
            lon_highest=lon
        elif lon < lon_lowest:
            lon_lowest=lon
            
        locations[f"{lat},{lon}"] = locations.get(f"{lat},{lon}",[lat,lon,0])
        locations[f"{lat},{lon}"][2] += 1
        
        ip_map.add_layer(Marker(title=ip["address"],location=(lat,lon)))
    except:
        pass

ip_map.fit_bounds([[lat_lowest,lon_lowest],[lat_highest,lon_highest]])
display(ip_map)